In [ ]:
# A0149963M 
# YSC2229 Assignment 2

In [9]:
def max_subarray_naive(arr, low, high):
    """
    arr - input array
    low - leftmost index (0)
    high - rightmost index (len(arr) - 1)
    
    Naive way of calculating the maximum subarray with
    O(n^2 time)
    """
    # base case if only 1 point in array
    if low == high:
        return(low, high, arr[low])
    
    # if there are >1 points in array
    maxSum = float('-inf')
    
    # for-loop that iterates all possibilites 
    for i in range(len(arr)):
        total = 0
        for j in range(i, len(arr)):
            total += arr[j]
            if maxSum < total :
                first = i
                last = j
                maxSum = total
                
    
    return (first, last, maxSum)


In [8]:
def max_crossing_subarray(arr, low, mid, high):
    """
    arr - input array
    low - leftmost index
    mid - index of midpoint in array
    high - rightmost index
    
    finds the maximum crossing subarray in an array in O(n)
    """
    
    # left subarray
    leftSum = float('-inf')
    maxLeft = float('-inf')
    total = 0
    for i in reversed(range(low, mid + 1)):
        total += arr[i]
        if total > leftSum:
            leftSum = total
            maxLeft = i
    
    # right subarray
    rightSum = float('-inf')
    maxRight = float('-inf')
    total = 0
    for j in range(mid + 1, high + 1):
        total += arr[j]
        if total > rightSum:
            rightSum = total
            maxRight = j
    return (maxLeft, maxRight, max(leftSum, max(rightSum, leftSum + rightSum)))


def max_subarray_eff(arr, low, high):
    """
    arr - input array
    low - leftmost index (0 for initialization)
    high - rightmost index (len(arr) - 1 for initialization)
    
    Efficient way of calculating maximum subarray in O(nlogn) time
    """
    
    if high == low:
        return(low, high, arr[low])
    else:
        mid = (low + high) // 2
        leftLow, leftHigh, leftSum = max_subarray_eff(arr, low, mid)
        rightLow, rightHigh, rightSum = max_subarray_eff(arr, mid + 1, high)
        crossLow, crossHigh, crossSum = max_crossing_subarray(arr, low, mid, high)
        
        if leftSum >= rightSum and leftSum >= crossSum:
            return (leftLow, leftHigh, leftSum)
        elif rightSum >= leftSum and rightSum >= crossSum:
            return (rightLow, rightHigh, rightSum)
        else: 
            return (crossLow, crossHigh, crossSum)
        

In [191]:
# testing
import random

def generate_random_array():
    """
    generates random array for testing purposes in the stress test.
    Just call this function during testing and compare function tested to
    another that is guaranteed to work.
    """
    l = random.randint(1, 100)
    out = []
    for i in range(l):
        out.append(random.randint(0,200) - 100)
    return out


def stress_test_subarray(fun1, fun2):
    """
    Stress test to test divide and conquer method for maximum subarray
    vs the naive implementation. This is my testing function to check that
    my implemementation works
    """
    for z in range(0, 2000):
        test_arr = generate_random_array()
        # 2nd index refers to the maximum subarray sum. Good test since maximum subarray can be 
        # non-unique, but not the sum of the maximum subarray.
        if fun2(test_arr, 0, len(test_arr) - 1)[2] != fun1(test_arr, 0, len(test_arr) - 1)[2]:
            print("Test failed for: ", test_arr)
            print("efficient algo output: ", fun2(test_arr, 0, len(test_arr) - 1))
            print("naive algo output: ", fun1(test_arr, 0, len(test_arr) - 1))
            return False
    return True
    
stress_test_subarray(max_subarray_naive, max_subarray_eff)

True

In [266]:
def dist(p1, p2): 
    """
    auxillary function to find distance between 2 points.
    """
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5  
  
def closest_naive(P, n): 
    """
    P - list containing points in the coordinate form (x, y)
    n - length of list (number of points, at least 2)
    
    Naive algorithm for finding closest points in a list in O(n^2).
    Emumerates all possibilities to find such a list
    """
    # distance set to infinite first and updated later
    min_val = float('inf') 
    
    # enumerates all possibilities to find minimum distance
    for i in range(n): 
        for j in range(i + 1, n): 
            if dist(P[i], P[j]) < min_val: 
                min_val = dist(P[i], P[j]) 
  
    return min_val 

def strip_closest(strip, size, d): 
    """
    strip - array obtained from closest_aux
    size - size of strip
    d - minimum distance
    
    Finds the minimum distance within the strip centered 
    at the midpoint of the points. Strip is already sorted
    on y-coordinates,
    """
    min_val = d  
    for i in range(size): 
        j = i + 1
        # while loop to compare distance of points within strip
        while j < size and strip[j][1] - strip[i][1] <= min_val: 
            min_val = min(min_val, dist(strip[i], strip[j]))
            j += 1
        
    return min_val  
  
def closest_aux(P, Q, n): 
    """
    P: [(x1, y1), ...] sorted by x-coordinates
    Q: [(x1, y1), ...] sorted by y-coordinates
    n: number of points in P (changes with recursive calls)
    """
    # base case where there are <= 3 points. Just use the naive algo
    # here since cost is minimal 
    if n <= 3:  
        return closest_naive(P, n) 
    
    mid = n // 2
    
    # find x-median point of array
    midPoint = P[mid] 
    
    # find minimal distance of left and right halves of array recusrively
    dl = closest_aux(P[:mid], Q, mid) 
    dr = closest_aux(P[mid:], Q, n - mid)  
    
    # find overall minimal distance between left and right halves of array
    d = min(dl, dr) 
    
    # strip creation of points with x-absolute distance of less than d 
    # from the midpoint
    strip = []  
    for i in range(n):  
        if abs(Q[i][0] - midPoint[0]) <= d: 
            strip.append(Q[i]) 
    
    # returns the minimum distance of left, right halves of points
    # and also the strip 

    return min(d, strip_closest(strip, len(strip), d)) 
  

def closest(P, n): 
    """
    n: number of points in P. n >= 2
    P: array containing points in the form [(x1, y1), ...]
    
    Main function to find closest n points
    """
    # sort according to x first 
    P.sort(key = lambda point: point[0]) 
    Q = P[:]
    # sort according to y second. Will be used for strip
    Q.sort(key = lambda point: point[1])      
    print(Q)
    return closest_aux(P, Q, n) 


In [268]:
# testing functions
import random

def generate_random_point():
    """
    Generates random point (x, y). 
    Used for testing
    """
    x = random.randint(0, 20) - 10
    y = random.randint(0, 20) - 10

    return (x, y)

def generate_random_array():
    # min number of pts is 2
    count = random.randint(2, 8)
    out = []
    for i in range(2, count + 1):
        out.append(generate_random_point())
    return out

def stress_test_points(fun1, fun2):
    for i in range(1000):
        test = generate_random_array()
        actual = fun1(test, len(test))
        expected = fun2(test, len(test))
        if actual != expected:
            print("Test failed for ", test)
            print("Test algorithm gave: ", actual)
            print("Witness algorithm gave: ", expected)
            return False
    return True

stress_test_points(closest, closest_naive)

[(-1, -5), (2, -2), (-4, 5), (-7, 6)]
[(-10, -6), (4, 1), (6, 2), (0, 7), (4, 7), (-10, 9)]
[(5, -8), (10, -5), (3, -2), (0, 8)]
[(-7, 0)]
[(6, -4), (-9, -2), (-8, -2), (-1, 4)]
[(3, -4), (-8, -2), (-6, 7), (-3, 7)]
[(0, -10), (-7, -8), (4, -7), (-8, -6), (-10, -1), (10, 1), (0, 3)]
[(-1, -6), (-6, 3)]
[(-2, 1), (0, 4)]
[(-10, 1), (9, 4)]
[(-4, 2), (9, 4), (-7, 9)]
[(8, -7), (6, -6), (9, -6), (1, -5), (5, 6)]
[(0, -3), (-3, 5)]
[(5, 4), (6, 9)]
[(-10, -1), (-4, 0), (3, 3), (7, 3), (-9, 7), (7, 7)]
[(-7, -3), (5, 5)]
[(8, -9), (-1, -8), (1, -4), (4, -1), (0, 7)]
[(-2, 5), (-6, 7), (-5, 9)]
[(-2, -9), (-5, 4), (-8, 6), (-6, 6)]
[(9, 3), (10, 8)]
[(5, -10), (-1, -6), (-4, -3), (-4, -1), (2, 1), (-3, 2), (7, 10)]
[(2, -3), (1, -1), (6, 2), (-5, 6), (5, 7), (10, 10)]
[(-9, -10), (-3, 6)]
[(1, -9), (10, -9), (-9, -2)]
[(-3, -8), (0, -7), (10, -3), (5, 3), (8, 4), (4, 7)]
[(9, -10), (-4, -7), (8, -2), (2, 0), (-7, 3), (10, 5), (10, 6)]
[(-7, -5), (9, 2), (-4, 4), (1, 9)]
[(-5, -8), (-7, 0), (

False